In [ ]:
import pandas as pd
import re

def label_comment(row):
    """Gắn nhãn bình luận theo các khía cạnh."""
    comment = str(row['cleaned_comment']).lower()
    row['vaccine_view'] = ''
    row['about_doctor_mike'] = ''
    row['format_criticism'] = ''
    row['misinformation'] = ''
    row['emotion_personal_experience'] = ''

    # Vaccine View
    if any(word in comment for word in ['vaccine', 'anti vax', 'vax', 'vaccination']):
        if any(word in comment for word in ['work', 'awesome', 'eradicated', 'safe', 'prevent']):
            row['vaccine_view'] = 'pro-vaccine'
        elif any(word in comment for word in ['dangerous', 'not safe', 'cause illness', 'death', 'autism']):
            row['vaccine_view'] = 'anti-vaccine'
        else:
            row['vaccine_view'] = 'skeptic'

    # About Doctor Mike
    if any(phrase in comment for phrase in ['dr mike', 'doctor mike', 'mike']):
        if any(word in comment for word in ['patient', 'respectful', 'kind', 'calm', 'eloquent']):
            row['about_doctor_mike'] = 'positive'
        elif any(word in comment for word in ['shill', 'joke', 'propaganda', 'indoctrinated', 'lie']):
            row['about_doctor_mike'] = 'negative'
        else:
            row['about_doctor_mike'] = 'neutral'

    # Format Criticism
    if any(word in comment for word in ['format', 'vote', 'flag', 'debate', 'system']):
        row['format_criticism'] = 'yes'

    # Misinformation
    if any(phrase in comment for phrase in [
        'hiv doesnt cause aids', 'autism', 'conspiracy', '600000 deaths', 'big pharma funded',
        'vaccines cause illness', 'flat earth', 'no placebo']):
        row['misinformation'] = 'yes'

    # Emotion/Personal Experience
    if any(word in comment for word in [
        'feel', 'sad', 'angry', 'pissed', 'frustrated', 'sorry', 'crazy', 'insane', 'painful',
        'my friend', 'my mum', 'my daughter', 'i lost']):
        row['emotion_personal_experience'] = 'yes'

    return row

def classify_aspects(df, column_name='cleaned_comment'):
    """Áp dụng gắn nhãn khía cạnh cho DataFrame."""
    return df.apply(label_comment, axis=1)

# Tải file
df = pd.read_csv('../data/raw/cleaned_comments.csv')

# Áp dụng phân loại khía cạnh
df = classify_aspects(df)

# Lưu kết quả
df.to_csv('../data/processed/labeled_comments.csv', index=False)

# Xem trước
print(df[['cleaned_comment', 'vaccine_view', 'about_doctor_mike', 'format_criticism',
          'misinformation', 'emotion_personal_experience']].head(10))

# Thống kê phân bố
print("\nPhân bố Vaccine View:")
print(df['vaccine_view'].value_counts())
print("\nPhân bố About Doctor Mike:")
print(df['about_doctor_mike'].value_counts())
print("\nPhân bố Format Criticism:")
print(df['format_criticism'].value_counts())
print("\nPhân bố Misinformation:")
print(df['misinformation'].value_counts())
print("\nPhân bố Emotion/Personal Experience:")
print(df['emotion_personal_experience'].value_counts())

In [2]:
import pandas as pd
from textblob import TextBlob

# Đọc file dữ liệu
df = pd.read_csv('../data/processed/labeled_comments.csv')

# Hàm phân tích cảm xúc để hỗ trợ đánh giá negative about_doctor_mike và emotion_personal_experience
def get_sentiment(comment):
    if pd.isna(comment):
        return 'neutral'
    analysis = TextBlob(comment)
    if analysis.sentiment.polarity < -0.3:  # Ngưỡng tiêu cực mạnh
        return 'strongly_negative'
    elif analysis.sentiment.polarity < 0:
        return 'negative'
    elif analysis.sentiment.polarity > 0:
        return 'positive'
    else:
        return 'neutral'

# Áp dụng phân tích cảm xúc
df['sentiment'] = df['cleaned_comment'].apply(get_sentiment)

# Hàm phân loại mức độ độc hại
def classify_toxicity(row):
    # Nếu tất cả các khía cạnh là NaN, xếp vào non-toxic
    if (pd.isna(row['vaccine_view']) and pd.isna(row['about_doctor_mike']) and
        pd.isna(row['format_criticism']) and pd.isna(row['misinformation']) and
        pd.isna(row['emotion_personal_experience'])):
        return 'non-toxic'

    # Độc hại cao: misinformation hoặc anti-vaccine
    if row['misinformation'] == 'yes' or row['vaccine_view'] == 'anti-vaccine':
        return 'highly-toxic'

    # Độc hại cao: about_doctor_mike negative với cảm xúc tiêu cực mạnh
    if row['about_doctor_mike'] == 'negative' and row['sentiment'] == 'strongly_negative':
        return 'highly-toxic'

    # Độc hại nhẹ: skeptic, format_criticism, negative about_doctor_mike (không mạnh), hoặc emotion_personal_experience
    if (row['vaccine_view'] == 'skeptic' or
        row['format_criticism'] == 'yes' or
        (row['about_doctor_mike'] == 'negative' and row['sentiment'] != 'strongly_negative') or
        row['emotion_personal_experience'] == 'yes'):
        return 'mildly-toxic'

    # Không độc hại: pro-vaccine, positive/neutral about_doctor_mike
    if (row['vaccine_view'] == 'pro-vaccine' or
        row['about_doctor_mike'] in ['positive', 'neutral']):
        return 'non-toxic'

    # Mặc định: non-toxic
    return 'non-toxic'

# Áp dụng phân loại
df['toxicity_level'] = df.apply(classify_toxicity, axis=1)

# In phân bố mức độ độc hại
print("Phân bố mức độ độc hại:")
print(df['toxicity_level'].value_counts(dropna=False))
print("\nTỷ lệ phần trăm mức độ độc hại:")
print(df['toxicity_level'].value_counts(normalize=True) * 100)

# Lưu DataFrame mới
df.to_csv('../data/processed/labeled_comments_with_toxicity.csv', index=False)

Phân bố mức độ độc hại:
toxicity_level
non-toxic       626
mildly-toxic    330
highly-toxic     44
Name: count, dtype: int64

Tỷ lệ phần trăm mức độ độc hại:
toxicity_level
non-toxic       62.6
mildly-toxic    33.0
highly-toxic     4.4
Name: proportion, dtype: float64
